In [ ]:
using Pkg
Pkg.add(["Oceananigans", "CairoMakie", "JLD2"])


In [6]:
using Oceananigans

# Define the computational grid
grid = RectilinearGrid(GPU(),size=(64, 64, 64), extent=(2π, 2π, 2π), topology=(Periodic, Periodic, Periodic))

# Create the model with WENO advection
model = NonhydrostaticModel(
    grid = grid,
    advection = WENO(order=5),             # 5th-order WENO scheme
    closure = ScalarDiffusivity(ν=1e-5),  # Small viscosity
    timestepper = :RungeKutta3       # 3rd-order Runge-Kutta time-stepping
)

println(model)


NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 64×64×64 RectilinearGrid{Float64, Periodic, Periodic, Periodic} on GPU with 3×3×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO reconstruction order 5
├── tracers: ()
├── closure: ScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-5)
├── buoyancy: Nothing
└── coriolis: Nothing


In [7]:
using Random, Statistics

# Extract velocity fields
u, v, w = model.velocities

# Generate random initial conditions
u₀ = rand(size(u)...)
v₀ = rand(size(v)...)
w₀ = rand(size(w)...)

# Ensure zero mean
u₀ .-= mean(u₀)
v₀ .-= mean(v₀)
w₀ .-= mean(w₀)

# Set the initial conditions
set!(model, u=u₀, v=v₀, w=w₀)


In [8]:
using Oceananigans.Utils: prettytime

# Create the simulation object
simulation = Simulation(model, Δt=0.1, stop_time=5)

# Configure adaptive time-stepping with TimeStepWizard
wizard = TimeStepWizard(cfl=0.4, max_Δt=0.2)
simulation.callbacks[:wizard] = Callback(wizard, IterationInterval(10))

# Progress logging
function progress(sim)
    @info "Iteration $(sim.model.clock.iteration): time = $(round(sim.model.clock.time, digits=2)) s, Δt = $(round(sim.model.clock.Δt, digits=4)) s"
end

simulation.callbacks[:logger] = Callback(progress, IterationInterval(50))


Callback of progress on IterationInterval(50)

In [11]:
using Printf

function progress(sim)
    max_u = maximum(abs, sim.model.velocities.u)
    max_v = maximum(abs, sim.model.velocities.v)
    max_w = maximum(abs, sim.model.velocities.w)
    max_velocity = maximum([max_u, max_v, max_w])
    @info "Iteration $(iteration(sim)), time = $(time(sim)), Δt = $(sim.Δt), max|u| = $(max_velocity)"
end

add_callback!(simulation, progress, IterationInterval(50))


In [12]:
using Oceananigans.Fields

# Vorticity components
ωx = ∂y(w) - ∂z(v)
ωy = ∂z(u) - ∂x(w)
ωz = ∂x(v) - ∂y(u)

# Vorticity magnitude
ω_mag = sqrt(ωx^2 + ωy^2 + ωz^2)

# Kinetic energy density
E = 0.5 * (u^2 + v^2 + w^2)


BinaryOperation at (Face, Center, Center)
├── grid: 64×64×64 RectilinearGrid{Float64, Periodic, Periodic, Periodic} on GPU with 3×3×3 halo
└── tree: 
    * at (Face, Center, Center)
    ├── 0.5
    └── + at (Face, Center, Center)
        ├── ^ at (Face, Center, Center)
        │   ├── 64×64×64 Field{Face, Center, Center} on RectilinearGrid on GPU
        │   └── 2
        ├── ^ at (Center, Face, Center)
        │   ├── 64×64×64 Field{Center, Face, Center} on RectilinearGrid on GPU
        │   └── 2
        └── ^ at (Center, Center, Face)
            ├── 64×64×64 Field{Center, Center, Face} on RectilinearGrid on GPU
            └── 2

In [13]:
simulation.output_writers[:fields] = JLD2OutputWriter(model, (; ω_mag, E),
                                                      schedule = TimeInterval(1.0),
                                                      filename = "3d_turbulence.jld2",
                                                      overwrite_existing = true)


JLD2OutputWriter scheduled on TimeInterval(1 second):
├── filepath: .\3d_turbulence.jld2
├── 2 outputs: (ω_mag, E)
├── array type: Array{Float64}
├── including: [:grid, :coriolis, :buoyancy, :closure]
├── file_splitting: NoFileSplitting
└── file size: 27.5 KiB

In [14]:
run!(simulation)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:184
┌ Info: Iteration 0, time = 0.0, Δt = 0.05, max|u| = 0.9497701241605511
└ @ Main c:\Users\dadoi\DataspellProjects\VS Code\Julia-Weather-Simulations\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl:8
┌ Info: Iteration 0, time = 0.0, Δt = 0.05, max|u| = 0.9497701241605511
└ @ Main c:\Users\dadoi\DataspellProjects\VS Code\Julia-Weather-Simulations\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl:8
┌ Info:     ... simulation initialization complete (23.874 seconds)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\M82LU\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (3.313 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia

In [17]:
using Oceananigans

# Load the time series
ω_mag_timeseries = FieldTimeSeries("3d_turbulence.jld2", "ω_mag")
E_timeseries = FieldTimeSeries("3d_turbulence.jld2", "E")

# Access times
times = ω_mag_timeseries.times

# Now you can proceed with your analysis or visualization
println("Loaded data at times:")
println(times)



Loaded data at times:
0.0:1.0:5.0


In [32]:
# Obtain grid coordinates from the field
xC, yC, zC = nodes(ω_mag_timeseries[1])

# Find the index of zC closest to z = π
z_index = findmin(abs.(zC .- π))[2]


64

In [33]:
println("zC: ", zC)


zC: [-6.283185307179586, -6.1850105367549055, -6.086835766330224, -5.988660995905543, -5.890486225480862, -5.792311455056181, -5.6941366846315, -5.595961914206819, -5.497787143782138, -5.399612373357457, -5.301437602932776, -5.203262832508095, -5.105088062083413, -5.006913291658733, -4.908738521234052, -4.81056375080937, -4.71238898038469, -4.614214209960009, -4.516039439535327, -4.417864669110647, -4.319689898685965, -4.221515128261284, -4.123340357836604, -4.025165587411922, -3.9269908169872414, -3.8288160465625602, -3.730641276137879, -3.6324665057131984, -3.5342917352885173, -3.436116964863836, -3.337942194439155, -3.2397674240144743, -3.141592653589793, -3.043417883165112, -2.945243112740431, -2.84706834231575, -2.748893571891069, -2.650718801466388, -2.5525440310417067, -2.454369260617026, -2.356194490192345, -2.2580197197676637, -2.1598449493429825, -2.061670178918302, -1.9634954084936207, -1.8653206380689396, -1.7671458676442586, -1.6689710972195775, -1.5707963267948966, -1.472

In [44]:
using CairoMakie

# Select a time index to visualize
time_index = length(times)  # Last time step

# Obtain grid coordinates from the field and convert to standard arrays
xC, yC, zC = nodes(ω_mag_timeseries[1])
xC = collect(xC)
yC = collect(yC)
zC = collect(zC)

# Find the index of zC closest to z = π
z_index = findmin(abs.(zC .- π))[2]

# Extract the 2D slice at z ≈ π and convert to standard array
ω_mag_slice = ω_mag_timeseries[time_index][:, :, z_index]
ω_mag_slice = collect(ω_mag_slice)

# Create the figure with updated keyword
fig = Figure(size = (800, 600))

# Create the axis
ax = Axis(fig[1, 1],
          xlabel = "x",
          ylabel = "y",
          title = "Vorticity Magnitude at z ≈ $(round(zC[z_index], digits=2)), time = $(round(times[time_index], digits=2))")

# Create the heatmap and capture the plot object
heatmap_plot = heatmap!(ax, xC, yC, ω_mag_slice';
                        colormap = :balance,
                        colorrange = (0, maximum(ω_mag_slice)))

# Add the colorbar, linking it to the heatmap plot
Colorbar(fig[1, 2], heatmap_plot)

# Display the figure
fig

# Save the figure
save("vorticity_magnitude.png", fig)


CairoMakie.Screen{IMAGE}
